In [1]:
import pandas as pd



# Nice code

In [49]:
form_resp = pd.read_csv('../participant data/Playlist Feedback Form (Responses) - Form Responses 1.csv')
playlist_data = pd.read_csv('../participant data/playlist_data.csv', index_col=0)
new_cols = [
    'ts',
    'email',
    'playlist_url',
    'num_of_songs',
    'satisfaction_1',
    'duration_listened',
    'satisfaction_2',
    'satisfaction_3',
    'matched_taste',
    'novelty',
    'songs_similar',
    'songs_diverse',
    'feedbackl_1',
    'feedback_2'
]
rename_dict = dict(zip(list(form_resp.columns), new_cols))
form_resp.rename(columns=rename_dict, inplace=True)

form_resp = pd.merge(form_resp, playlist_data, how='inner', on = 'email')
form_resp["is_diverse"] = form_resp.playlist_url == form_resp.diverse
form_dict = form_resp.to_dict('records')
df = pd.read_csv('../participant data/survey data/msi_response.csv')
table = df.groupby(['user_id', 'item_id', 'value']).sum().reset_index()
tab = table.set_index('user_id')

usr_info_list = []

for row in form_dict:
    USR = row['user_id']

    usr = tab.loc[USR]
    d = dict(zip(usr.item_id, usr.value))
    [d.pop(k) for k in ['8', '9', 'Emotions[16]', 'email']]

    negative_A = 'Active Engagement[5]'
    negative_E = 'Emotions[11]'
    positive_A = [int(d[k]) for k in list(d.keys()) if k != negative_A and k.startswith('A')]
    positive_E = [int(d[k]) for k in list(d.keys()) if k != negative_E and k.startswith('E')]

    active_eng = float(sum(positive_A) + ( 8 - int(d[negative_A]))) / 7
    emotions = float(sum(positive_E) + ( 8 - int(d[negative_E]))) / 6

    satisfaction = float(sum([row[k] for k in row.keys() if k.startswith('satisfaction')])) / 3

    diversity = float( row['songs_diverse'] + (6 - row['songs_similar'])) / 2

    end_dict = {
        'user_id' : USR,
        'is_diverse' : row['is_diverse'],
        'active_eng' : active_eng,
        'emotions' : emotions,
        'satisfaction' : satisfaction,
        'diversity' : diversity,
        'matched_taste' : row['matched_taste'],
        'novelty' : row['novelty'],
    }
    usr_info_list.append(end_dict)

out_df = pd.DataFrame(usr_info_list)

out_df

,user_id,is_diverse,active_eng,emotions,satisfaction,diversity,matched_taste,novelty
0,ef5892d90c5522ca63c291c8553d377a70104d8559a282...,True,3.428571,5.666667,4.000000,3.5,4,4
1,ef5892d90c5522ca63c291c8553d377a70104d8559a282...,False,3.428571,5.666667,2.000000,2.5,2,5
2,139fe82c44eb2ccdc92e6cf99e6ed512414efe88d5b965...,True,5.714286,6.500000,3.333333,3.5,5,4
3,811dc65452d1be20e073f385ee6f0df18ee97d54f8edc1...,True,4.000000,5.833333,3.666667,2.0,4,2
4,811dc65452d1be20e073f385ee6f0df18ee97d54f8edc1...,False,4.000000,5.833333,2.333333,4.0,3,4
5,7168a37151da4adb963f8b79981071575b3e1d56c973c9...,False,3.571429,4.666667,3.333333,5.0,4,4
6,7168a37151da4adb963f8b79981071575b3e1d56c973c9...,True,3.571429,4.666667,2.666667,5.0,2,4
